<a href="https://colab.research.google.com/github/zkhandker/rupi-kaur/blob/main/Zuhair01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2.1 Dependencies 


In [29]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 

# Download and import the MIT 6.S191 package
!pip install mitdeeplearning
import mitdeeplearning as mdl

# Import all remaining packages
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
!apt-get install abcmidi timidity > /dev/null 2>&1

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
## assert len(tf.config.list_physical_devices('GPU')) > 0
import pathlib

## 2.1 Load Poems 


In [30]:
#data_url = 'https://raw.githubusercontent.com/zkhandker/rupi-kaur/main/data/test.txt'
#dataset = tf.keras.utils.get_file('test.txt',
#    data_url)
#dataset_dir = pathlib.Path(dataset).parent

In [ ]:
# tf.io.gfile.remove(dataset_dir/'test2.txt')

In [31]:
#list(dataset_dir.iterdir())

In [32]:
#sample_file = dataset_dir/'test.txt'
#with open(sample_file) as f:
  #print(f.read())
#  sentence=f.read()
#  print(sentence)

In [23]:
path_to_file = tf.keras.utils.get_file('combined2', 'https://raw.githubusercontent.com/zkhandker/rupi-kaur/main/data/combined2.txt')
with open(path_to_file) as f: 
  poems = f.read()
  #print(poems)

73728/66843 [=================================] - 0s 0us/step


In [24]:
#sentence = "The wide road shimmered in the hot sun"
tokens = list(poems.lower().split())
print(len(tokens))

13374


In [26]:
vocab, index = {}, 1 # start indexing from 1
vocab['<pad>'] = 0 # add a padding token 
for token in tokens:
  if token not in vocab: 
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)
print(vocab_size)

{'<pad>': 0, 'how': 1, 'is': 2, 'it': 3, 'so': 4, 'easy': 5, 'for': 6, 'you': 7, 'to': 8, 'be': 9, 'kind': 10, 'people': 11, 'he': 12, 'asked': 13, 'milk': 14, 'and': 15, 'honey': 16, 'dripped': 17, 'from': 18, 'my': 19, 'lips': 20, 'as': 21, 'i': 22, 'answered': 23, 'cause': 24, 'have': 25, 'not': 26, 'been': 27, 'me': 28, 'the': 29, 'first': 30, 'boy': 31, 'that': 32, 'kissed': 33, 'held': 34, 'shoulders': 35, 'down': 36, 'like': 37, 'handlebars': 38, 'of': 39, 'bicycle': 40, 'ever': 41, 'rode': 42, 'was': 43, 'five': 44, 'had': 45, 'smell': 46, 'starvation': 47, 'on': 48, 'his': 49, 'which': 50, 'picked': 51, 'up': 52, 'father': 53, 'feasting': 54, 'mother': 55, 'at': 56, '4': 57, 'a.m.': 58, 'teach': 59, 'body': 60, 'giving': 61, 'those': 62, 'wanted': 63, 'should': 64, 'feel': 65, 'anything': 66, 'less': 67, 'than': 68, 'whole': 69, 'god': 70, 'did': 71, 'empty': 72, '4:25': 73, "i've": 74, 'sex': 75, 'she': 76, 'said': 77, 'but': 78, "don't": 79, 'know': 80, 'what': 81, 'making':

In [27]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'how', 2: 'is', 3: 'it', 4: 'so', 5: 'easy', 6: 'for', 7: 'you', 8: 'to', 9: 'be', 10: 'kind', 11: 'people', 12: 'he', 13: 'asked', 14: 'milk', 15: 'and', 16: 'honey', 17: 'dripped', 18: 'from', 19: 'my', 20: 'lips', 21: 'as', 22: 'i', 23: 'answered', 24: 'cause', 25: 'have', 26: 'not', 27: 'been', 28: 'me', 29: 'the', 30: 'first', 31: 'boy', 32: 'that', 33: 'kissed', 34: 'held', 35: 'shoulders', 36: 'down', 37: 'like', 38: 'handlebars', 39: 'of', 40: 'bicycle', 41: 'ever', 42: 'rode', 43: 'was', 44: 'five', 45: 'had', 46: 'smell', 47: 'starvation', 48: 'on', 49: 'his', 50: 'which', 51: 'picked', 52: 'up', 53: 'father', 54: 'feasting', 55: 'mother', 56: 'at', 57: '4', 58: 'a.m.', 59: 'teach', 60: 'body', 61: 'giving', 62: 'those', 63: 'wanted', 64: 'should', 65: 'feel', 66: 'anything', 67: 'less', 68: 'than', 69: 'whole', 70: 'god', 71: 'did', 72: 'empty', 73: '4:25', 74: "i've", 75: 'sex', 76: 'she', 77: 'said', 78: 'but', 79: "don't", 80: 'know', 81: 'what', 82: 'maki

In [35]:
vectorized_poems = np.array([vocab[word] for word in tokens])
print(vectorized_poems)

[   1    2    3 ... 1490   22 1491]


In [36]:
print ('{} ---- characters mapped to int ----> {}'.format(repr(poems[:10]), vectorized_poems[:10]))
# check that vectorized_poems is a numpy array
assert isinstance(vectorized_poems, np.ndarray), "returned result should be a numpy array"

'how is it ' ---- characters mapped to int ----> [ 1  2  3  4  5  6  7  8  9 10]


In [38]:
### Batch definition to create training examples ###

def get_batch(vectorized_songs, seq_length, batch_size):
  # the length of the vectorized songs string
  n = vectorized_songs.shape[0] - 1
  # randomly choose the starting indices for the examples in the training batch
  idx = np.random.choice(n-seq_length, batch_size)

  '''TODO: construct a list of input sequences for the training batch'''
  input_batch = [vectorized_songs[i : i+seq_length] for i in idx]
  # input_batch = # TODO
  '''TODO: construct a list of output sequences for the training batch'''
  output_batch = [vectorized_songs[i+1 : i+seq_length+1] for i in idx]
  # output_batch = # TODO

  # x_batch, y_batch provide the true inputs and targets for network training
  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])
  return x_batch, y_batch


# Perform some simple tests to make sure your batch function is working properly! 
test_args = (vectorized_poems, 10, 2)
if not mdl.lab1.test_batch_func_types(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_shapes(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_next_step(get_batch, test_args): 
   print("======\n[FAIL] could not pass tests")
else: 
   print("======\n[PASS] passed all tests!")

[PASS] test_batch_func_types
[PASS] test_batch_func_shapes
[PASS] test_batch_func_next_step
[PASS] passed all tests!


In [41]:
x_batch, y_batch = get_batch(vectorized_poems, seq_length=5, batch_size=1)

In [43]:
print(x_batch)
print(y_batch)

[[327 158 416 417 194]]
[[158 416 417 194 226]]
